<a href="https://colab.research.google.com/github/ptran1203/style_transfer/blob/master/Adain_Style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
from google.colab import drive, output
data_loaded = False
drive.mount('/content/drive')
BASE_DIR = "/content/drive/My Drive/Style_Transfer"
!rm -rf '/content/style_transfer'
!git clone https://github.com/ptran1203/style_transfer

In [ ]:
cd style_transfer

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pickle
import os
import utils
from dataloader import DataGenerator
from model import *

class DataGen(DataGenerator):
    BATCH_FILES= 4

class SModel(StyleTransferModel):
    pass


style_layer_names=[
    'block1_conv1', 'block2_conv1',
    'block3_conv1', 'block4_conv1',
]
last_layer='block4_conv1'
pre_trained_model = 'vgg19'
rst = 256
data_gen = DataGen(BASE_DIR, 8, rst=rst, max_size=1500, multi_batch=False,
                   normalize=True)
smodel = SModel(BASE_DIR, None, 1e-4,
                style_layer_names=style_layer_names,
                last_layer=last_layer, 
                show_interval=5,
                style_loss_weight=3.5,
                pre_trained_model=pre_trained_model)

In [ ]:
smodel.load_weight()
smodel.train(data_gen, 500, augment_factor=0)

In [ ]:
urls = [
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/escher_sphere_thumb.jpg',
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/udnie_thumb.jpg',
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/mosaic_thumb.jpg',
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/raw/master/images/style_thumb/cat_thumb.jpg'
    'https://github.com/lengstrom/fast-style-transfer/blob/master/examples/style/rain_princess.jpg?raw=true',
    'https://github.com/lengstrom/fast-style-transfer/blob/master/examples/style/wave.jpg?raw=true',
]
cimg = utils.http_get_img(
    'https://yt3.ggpht.com/a/AATXAJx3V2SYpa27ubB-eIw_vzBgS1QHKcBGj5xAZZ7dQQ=s900-c-k-c0xffffffff-no-rj-mo',
    # 'https://github.com/elleryqueenhomels/arbitrary_style_transfer/blob/master/images/content/stata.jpg?raw=true',
    512
)
cv2_imshow(utils.deprocess(utils.de_norm(cimg[0])))
for url in urls:
    simg = utils.http_get_img(url, 512)
    gen = smodel.generate(cimg, simg)
    
    cv2_imshow(utils.deprocess(utils.de_norm(simg[0])))
    cv2_imshow(utils.deprocess(utils.de_norm(gen[0])))